<a href="https://colab.research.google.com/github/hanochk/myRepo/blob/master/load_trt_plan_file_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and infer a TRT plan file from a TF-TRT converted model (TF2)
This notebook demonstrates how to load a TensorRT plan file from a TF-TRT converted model and run TRT inference with the plan file. The plan file is a serialized TRT engine.

**Note:** This example assumes that the whole model is converted to a single TRT engine. In that case the single plan file can be used to infer the model using TRT's C++ or Python API.

To run this notebook, a recent TF version is necessary, for example the [NGC docker container](https://ngc.nvidia.com/catalog/containers/nvidia:tensorflow) nvcr.io/nvidia/tensorflow:20.12-tf2-py3

In [ ]:
# Verbose debug output about the TF-TRT conversion
import os
os.environ["TF_CPP_VMODULE"]="convert_graph=1,segment=1"

Might need to install some packages

In [ ]:
!pip install pillow matplotlib

In [ ]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import numpy as np
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.framework import convert_to_constants
from tensorflow.python.ops import gen_math_ops
import matplotlib.pyplot as plt

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## 1. Load and save the TF2 model

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions

model = ResNet50V2(weights='imagenet') 
tf.saved_model.save(model, 'resnet_v2_50_saved_model')

Define model input parameters

In [ ]:
img_shape = [224, 224, 3]
batch_size = 1
input_shape = [batch_size,] + img_shape

### 1.1 Get input data

In [ ]:
!mkdir data
!wget  -O ./data/img0.JPG "https://d17fnq9dkz9hgj.cloudfront.net/breed-uploads/2018/08/siberian-husky-detail.jpg?bust=1535566590&width=630"

In [ ]:
img_path = './data/img0.JPG'
input_img = image.load_img(img_path, target_size=(224, 224))
plt.imshow(input_img);
plt.axis('off');

Preprocess the data for the model.

In [ ]:
input_x = image.img_to_array(input_img)
input_x = np.expand_dims(input_x, axis=0)
input_x = preprocess_input(input_x)

## 2. Convert model to TF-TRT

In [ ]:
conv_params=trt.TrtConversionParams(precision_mode='FP16', max_workspace_size_bytes=1<<30)
converter = trt.TrtGraphConverterV2(input_saved_model_dir="resnet_v2_50_saved_model", conversion_params=conv_params)
converter.convert()

def input_fn():
    input_shapes = [[input_x.shape], ]       
    for shapes in input_shapes:
        # return a list of input tensors
        yield [np.ones(shape=x).astype(np.float32) for x in shapes]
        
converter.build(input_fn)
converter.save("resnet_v2_50_trt")
print("Model exported to TRT")

### 2.1 Load and test converted model

In [ ]:
def get_func_from_saved_model(saved_model_dir):
    saved_model_loaded = tf.saved_model.load(
        saved_model_dir, tags=[tag_constants.SERVING])
    graph_func = saved_model_loaded.signatures[
        signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
    return graph_func, saved_model_loaded

In [ ]:
trt_func, _ = get_func_from_saved_model('resnet_v2_50_trt')

Run prediction

In [ ]:
preds = trt_func(tf.convert_to_tensor(input_x))['predictions'].numpy()

The tensor `preds` stores the predicted class probabilities for the single input image in the batch.

In [ ]:
decode_predictions(preds, top=3)[0]

## 3. Load engine file (plan file) directly to TensorRT
The converted model is saved under `resnet_v2_50_trt`. The TensorRT engines can be found in the `assets` subdirectory. Each TRTEngineOp has a separate asset file:

In [ ]:
import numpy as np
import tensorflow as tf

# This is the protobuf that is needed to load the TF plan file
from tensorflow.compiler.tf2tensorrt.utils.trt_engine_instance_pb2 import TRTEngineInstance

In [ ]:
# Check if the file is where we expect it to be
!ls -lh resnet_v2_50_trt/assets

In [ ]:
asset_file = 'resnet_v2_50_trt/assets/trt-serialized-engine.TRTEngineOp_0_0'

### 3.1 Load engine file (plan file)

In [ ]:
def read_plan_from_asset(asset_file):
    """ Read the serialized TRT engine from an asset file, and return the buffer that contains the first engine.
    Arguments: asset_file -- path to the file
    """
    raw_dataset = tf.data.TFRecordDataset([asset_file])

    # Note that the asset file could contain multiple engines, one for each input shape that the
    # corresponding TRTEngineOp handles. Here we only return the buffer that contains the first engine.
    for raw_record in raw_dataset.take(1):
        engine_instance = TRTEngineInstance()
        engine_instance.ParseFromString(raw_record.numpy())
        # print("Loaded engine for shape", engine_instance.input_shapes)
        
    return engine_instance.serialized_engine

In [ ]:
engine_string = read_plan_from_asset(asset_file)

### 3.2 Deserialize the engine and inspect it
First we load the necessary modules.

In [ ]:
!pip install pycuda # can take a few minutes
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt
logger = trt.Logger()
runtime = trt.Runtime(logger)

In [ ]:
with runtime.deserialize_cuda_engine(engine_string) as engine:
    print('Engine bindings:')
    for i in range(engine.num_bindings):
        print('Name:',engine[i], ', shape: ', engine.get_binding_shape(i))

Since the whole model was converted to this engine, the input and output bindings correspond to the TF model's input and output tensors. If the binding shapes do not match the TF model input/output shapes, then the engine contains only a part of the graph.

### 3.3 Write the engine file into a standalone binary file
In case someone prefers plain binary files. This is not used in this notebook, just provided as reference.

In [ ]:
with open("/tmp/plan_file.trt","wb") as f:
    f.write(engine_string)

## 4 Native TensorRT inference in Python
Define helper functions:

In [ ]:
# Simple helper data class that's a little nicer to use than a 2-tuple.
class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
    
# Allocates all buffers required for an engine, i.e. host/device inputs/outputs.
def allocate_buffers(engine):
    inputs = []
    outputs = []
    bindings = []
    stream = cuda.Stream()
    for binding in engine:
        size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
        dtype = trt.nptype(engine.get_binding_dtype(binding))
        # Allocate host and device buffers
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        # Append the device buffer to device bindings.
        bindings.append(int(device_mem))
        # Append to the appropriate list.
        if engine.binding_is_input(binding):
            inputs.append(HostDeviceMem(host_mem, device_mem))
        else:
            outputs.append(HostDeviceMem(host_mem, device_mem))
    return inputs, outputs, bindings, stream

def do_inference(context, bindings, inputs, outputs, stream, batch_size=1):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]

Run inference

In [ ]:
with runtime.deserialize_cuda_engine(engine_string) as engine:
    allocate_buffers(engine)
    inputs, outputs, bindings, stream = allocate_buffers(engine)
    with engine.create_execution_context() as context:
        # Set input data
        np.copyto(inputs[0].host, input_x.ravel())
        [output] = do_inference(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)

In [ ]:
output = output.reshape(batch_size, len(output)//batch_size)

In [ ]:
decode_predictions(output, top=3)[0]